In [1]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


%matplotlib inline
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import json

In [2]:
train = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100, 102, 104, 106, 108, 110, 112, 114, 116, 118, 120, 122, 124, 126, 128, 130, 132, 134, 136, 138, 140, 142, 144, 146, 148, 150, 152, 154, 156, 158, 160, 162, 164, 166, 168, 170, 172, 174, 176, 178, 180, 182, 184, 186, 188, 190, 192, 194, 196, 198, 200, 202]
validation =  [3, 7, 11, 15, 19, 23, 27, 31, 35, 39, 43, 47, 51, 55, 59, 63, 67, 71, 75, 79, 83, 87, 91, 95, 99, 103, 107, 111, 115, 119, 123, 127, 131, 135, 139, 143, 147, 151, 155, 159, 163, 167, 171, 175, 179, 183, 187, 191, 195, 199]
test = [5, 9, 13, 17, 21, 25, 29, 33, 37, 41, 45, 49, 53, 57, 61, 65, 69, 73, 77, 81, 85, 89, 93, 97, 101, 105, 109, 113, 117, 121, 125, 129, 133, 137, 141, 145, 149, 153, 157, 161, 165, 169, 173, 177, 181, 185, 189, 193, 197, 201]


In [8]:
acc_data = pd.read_csv('eval_acc.csv')
test_data = acc_data.iloc[test]
test_data = test_data.dropna()
#rows with nan values dropped


In [9]:
class MyModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MyModel, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, 2,batch_first=True)
        self.fc = nn.Linear(hidden_size, 32)
        self.fc2 = nn.Linear(32,16)
        self.fc3 = nn.Linear(16,output_size)
    
    def forward(self, input):
        batch_size = input.size(0)
        hidden = self.init_hidden(batch_size)
        output, hidden = self.rnn(input, hidden)
        #print(output,hidden)
        output = nn.ReLU()(self.fc(output))
        output = nn.ReLU()(self.fc2(output))
        output = self.fc3(output)
        return output
    
    def init_hidden(self, batch_size):
        '''
        To initialize hidden state
        '''
        hidden = torch.zeros(2, batch_size, self.hidden_size)  
        return hidden


input_size = 9
#hidden_size = 64
hidden_size = 12
output_size = 1
# sequence_length = 9
batch_size = 32  

model = MyModel(input_size, hidden_size, output_size)

In [10]:
model.load_state_dict(torch.load('/content/new_best_model_rmse.pth'))

<All keys matched successfully>

In [17]:
## can change the values of E and M, E>M minimum value of E = 5

E = 10
M = 150

task_set = test_data.iloc[:,0:4+E]
fixed = torch.Tensor(task_set.iloc[:,0:4].values)
sequence = torch.Tensor(task_set.iloc[:,-5:].values)

In [22]:
model.eval()
with torch.no_grad():
  for i in range(E+1,M+1):
      new_pred = model(torch.cat((fixed.unsqueeze(1),sequence.unsqueeze(1)),dim = 2))
      sequence = torch.cat((sequence[:,1:],new_pred.squeeze(2)),dim = 1)

In [24]:
# output for all the test set
# to accomodate all the values of E without errors for the user, rows with nan values are dropped
print(new_pred.squeeze(1).squeeze(1))

tensor([0.4732, 0.4262, 0.7145, 0.5011, 0.4287, 1.0939, 0.8820, 0.6598, 1.1572,
        1.1425, 0.9642, 1.1614, 1.1853, 1.1883, 0.6980, 0.5387, 0.5623, 0.7491,
        0.5401, 0.5528, 0.8386, 0.5985, 0.6016, 0.5529, 0.6018, 0.6216, 0.6126,
        0.6027, 0.6236, 0.6146, 0.6052, 0.6301, 0.6212, 0.6066, 0.6345, 0.6258,
        0.6083, 0.6471, 0.6405, 0.6069, 0.6549, 0.6503, 0.5600, 0.6848, 0.7009,
        0.6037, 0.6897, 0.5785, 0.6896])
